In [34]:
import hashlib

def sha256_hash(data, nonce):
    # Concatenate the data and nonce and encode as bytes
    input_str = f"{data}{nonce}".encode()
    # Compute the SHA-256 hash
    hash_hex = hashlib.sha256(input_str).hexdigest()
    # Convert the hash to an integer for comparison
    return int(hash_hex, 16)


In [35]:
import hashlib
def simple_hash(data, nonce):
    return (data + nonce) % 10  # Simplified hash function
def classical_mining(data, target_zeros):
    nonce = 0
    steps = 0
    target_prefix = '0' * target_zeros
    while True:
        steps += 1
        hash_value = sha256_hash(data, nonce)
        if str(format(hash_value, '0256b'))[:target_zeros] == target_prefix:
            return nonce, steps
        nonce += 1

# Example usage
data = 123790790  # Some arbitrary data
target = 10  # We want the hash to be 0
nonce = classical_mining(data, target)
print(f"Classical mining found nonce: {nonce}")


Classical mining found nonce: (2859, 2860)


In [ ]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCMT, GroverOperator
import time
import hashlib
from math import sqrt, pi
from qiskit.quantum_info import Operator
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.tools.monitor import job_monitor

def grovers_oracle(target_nonce, num_qubits):
    oracle = QuantumCircuit(num_qubits)
    # Apply X gates to flip the qubits corresponding to the target nonce
    for i in range(num_qubits):
        if (target_nonce >> i) & 1:
            oracle.x(i)
    # Apply a multi-controlled X gate to flip the sign of the target state
    oracle.h(num_qubits - 1)
    oracle.append(MCMT('x', num_ctrl_qubits=num_qubits - 1, num_target_qubits=1), list(range(num_qubits)))
    oracle.h(num_qubits - 1)
    # Flip the qubits back
    for i in range(num_qubits):
        if (target_nonce >> i) & 1:
            oracle.x(i)
    return oracle

def grovers_diffuser(num_qubits):
    diffuser = QuantumCircuit(num_qubits)
    diffuser.h(range(num_qubits))
    diffuser.x(range(num_qubits))
    diffuser.h(num_qubits - 1)
    diffuser.append(MCMT('x', num_ctrl_qubits=num_qubits - 1, num_target_qubits=1), list(range(num_qubits)))
    diffuser.h(num_qubits - 1)
    diffuser.x(range(num_qubits))
    diffuser.h(range(num_qubits))
    return diffuser

def quantum_mining(data, target, num_qubits=3):
    target_nonce = classical_mining(data, target)[0]  # Get target nonce from classical mining
    oracle = grovers_oracle(target_nonce, num_qubits)
    diffuser = grovers_diffuser(num_qubits)

    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))  # Initialize in superposition

    num_steps = int(np.pi / 4 * np.sqrt(2 ** num_qubits))

    for _ in range(num_steps):
        qc.append(oracle, range(num_qubits))
        qc.append(diffuser, range(num_qubits))

    qc.measure(range(num_qubits), range(num_qubits))

    backend = AerSimulator()
    transpiled_circuit = transpile(qc, backend)
    job = backend.run(transpiled_circuit, shots=1)  # Only need one shot for verification
    result = job.result()
    counts = result.get_counts(qc)
    measured_nonce = int(max(counts, key=counts.get), 2)  # Convert binary string to integer

    # Verify that the measured nonce is correct
    if simple_hash(data, measured_nonce) == target:
        print(f"Quantum mining found the correct nonce: {measured_nonce}")
    else:
        print(f"Quantum mining did not find the correct nonce. Measured nonce: {measured_nonce}")

    return counts, num_steps




# Classical mining
nonce, classical_steps = classical_mining(data, target)
print(f"Classical mining found nonce: {nonce} in {classical_steps} steps")

# Quantum mining
counts, quantum_steps = quantum_mining(data, target, num_qubits)
print(f"Quantum mining results: {counts}")
print(f"Quantum mining took {quantum_steps} steps")

#

Classical mining found nonce: 7 in 7 steps
Quantum mining results: {'101': 6, '110': 9, '001': 9, '100': 8, '011': 9, '111': 10, '010': 9, '000': 964}
Quantum mining took 2 steps
